In [3]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET 
import pandas as pd
from geopy.geocoders import Nominatim
import re

## Reading Data

In [37]:
dlrPitches = pd.read_csv('data/dlr-pitches.csv')

In [38]:
dccPitches = pd.read_csv('data/dccplayingpitchesp20120816-1550.csv')

In [53]:
xtree = ET.parse("data/fccplayingpitchesp20111203-1424.xml")
xroot = xtree.getroot()
cols = pd.Series(['FACILITY_TYPE','FACILITY_NAME','LOCATION','LAT','LONG'])
fccPitches = pd.DataFrame(columns = cols)
for node in xroot[0]: 
    ft = node.find("FACILITY_TYPE").text
    fn = node.find("FACILITY_NAME").text
    loc = node.find("LOCATION").text
    lat = node.find("LAT").text
    lo= node.find("LONG").text
    temp = [ft,fn,loc,lat,lo]
    fccPitches = fccPitches.append(pd.Series(temp, index = cols), ignore_index = True)

## Data Observation and Modelling

**We use [head] to take a look at the three data sources** 

In [40]:
dlrPitches.head()

,Location,Number,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,NaN,2,SSG,53.257614,-6.139882
2,NaN,3,SSG,53.257842,-6.139265
3,NaN,4,SSG,53.257098,-6.139094
4,NaN,5,SGG,53.256674,-6.140134


In [41]:
dlrPitches[dlrPitches.Location.isnull()].index.size

46

what is Number?

In [42]:
dlrPitches.Number.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 'A', 'B',
       'All weather', '12', '13', '14', 'GAA A', 'GAA B', 'Soccer', 'GAA',
       'Temporary pitch', 'Rugby', ' GAA ', 'Location of future pitchs',
       'A ', nan], dtype=object)

In [19]:
dccPitches.head()

,PARK,AREA,CLUBNAME,LEAGUE,Unnamed: 4
0,ALBERT COLLEGE,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE,NaN
1,ALBERT COLLEGE,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE,NaN
2,BEECHILL,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE,NaN
3,BELCAMP,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE,NaN
4,BELCAMP,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE,NaN


In [52]:
fccPitches.head()

,Facility_Type,Facility_Name,Location,Lat,Long
0,All weather pitches,Balbriggan Town Park,Balbriggan,53.6049596246817,-6.18235291959051
1,All weather pitches,Balheary Reservoir,Swords,53.4727096370551,-6.22301521551813
2,All weather pitches,Town Park,Skerries,53.5771135903791,-6.11107205744599
3,All weather pitches,St. Mologa's Park,Balbriggan,53.6176672458903,-6.18936794084573
4,Basketball Court,Seagrange Park,None,53.3966674985382,-6.13535180348378


In [21]:
fccPitches.FACILITY_TYPE.unique()

array(['All weather pitches', 'Basketball Court', 'GAA Pitches',
       'Rugby Pitches', 'Seven-a-side', 'Soccer pitches', 'Tennis Courts'],
      dtype=object)

### Observations:
#### DLR
1. has many empty locations (46 out of 61) which are assumed from the previous row
2. column "Number" represents facility type >rename
3. we get more info about location using the coordinations

#### DCC 
1. has no coordinates > we use an outer source to get this info based on the location provided
2. content is all upper case > change to lower case
3. It is only for Soccer (football) pitches = Facility type

#### FCC
1. has coordinates and locations 

the three data sources have the name of the park in different ways (Location, Park, Facility Name)> 

1. In DCC change Park to Location
2. In FCC change the fields "location" to "Town & Lat, Long to latitude, longitude & Facitlity Name to Location
3. Lower case all columns names 

---------------------

### Modelling
Final data source will have the following columns:
* Location
* Latitude
* Longitude
* Address 
* Facility Type

extra field (not for everyone)
* Size
* Club Name
* League


### DLR

In [140]:
# DLR has many empty locations which are assumed from the previous row
enriched_dlr=pd.DataFrame(dlrPitches).fillna(method='ffill')
enriched_dlr.head()

,Location,Number,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,Kilbogget Park,2,SSG,53.257614,-6.139882
2,Kilbogget Park,3,SSG,53.257842,-6.139265
3,Kilbogget Park,4,SSG,53.257098,-6.139094
4,Kilbogget Park,5,SGG,53.256674,-6.140134


In [141]:
#getting address info
enriched_dlr["cords"] = enriched_dlr["Latitude"].map(str) +","+ enriched_dlr["Longitude"].map(str)
enriched_dlr['Address']  = enriched_dlr["cords"].apply(lambda x:geolocator.reverse(x).address) 


In [142]:
enriched_dlr.drop(columns="cords",inplace=True)

In [143]:
enriched_dlr.head()

,Location,Number,Size,Latitude,Longitude,Address
0,Kilbogget Park,1,Snr,53.257242,-6.140665,"Running track, Shanganagh Vale, Loughlinstown,..."
1,Kilbogget Park,2,SSG,53.257614,-6.139882,"Seapoint Rugby Club, Churchview Road, Kilbogge..."
2,Kilbogget Park,3,SSG,53.257842,-6.139265,"Seapoint Rugby Club, Churchview Road, Kilbogge..."
3,Kilbogget Park,4,SSG,53.257098,-6.139094,"63, Coolevin, Gleann na Sailí, Loughlinstown, ..."
4,Kilbogget Park,5,SGG,53.256674,-6.140134,"Running track, Shanganagh Vale, Loughlinstown,..."


In [145]:
# duplicates
enriched_dlr[enriched_dlr.duplicated(["Address","Size","Number"])]

,Location,Number,Size,Latitude,Longitude,Address
50,Sallynoggin,All weather,mini,53.273968,-6.144321,"Pearse Park, Thomastown, Dun Laoghaire-Sallyno..."
51,Sallynoggin,All weather,mini,53.273792,-6.144107,"Pearse Park, Thomastown, Dun Laoghaire-Sallyno..."


In [146]:
enriched_dlr.drop_duplicates(subset=["Address","Size","Number"],inplace=True)

In [147]:
enriched_dlr.rename(columns={"Number":"Facility_Type"},inplace=True)

### DCC 

In [47]:
# DCC has no coordinates > we use an outer source to get this info based on the location provided

# remove empty field
enriched_dcc= pd.DataFrame(dccPitches.iloc[:, :-1])

# use Nominatim to get coordinates
geolocator = Nominatim()


# call geocode and get Latitude and longitude
# add Dublin Ireland to the location to prevent ambuguity
enriched_dcc['city_coord']  = enriched_dcc['PARK'].apply(lambda x:geolocator.geocode(x+",Dublin,Ireland")) 


enriched_dcc['Latitude'] = enriched_dcc['city_coord'].apply(lambda x:  x.latitude if x != None else 0)
enriched_dcc['Longitude'] = enriched_dcc['city_coord'].apply(lambda x:  x.longitude if x != None else 0)
enriched_dcc['Address'] = enriched_dcc['city_coord'].apply(lambda x:  x.address if x != None else 0)
#remove extra column city coord
enriched_dcc.drop(columns="city_coord",inplace=True)

/Users/suaddarra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  import sys


In [76]:
enriched_dcc.drop(columns="AREA",inplace=True)

In [77]:
enriched_dcc.tail()

,Location,Clubname,League,Latitude,Longitude,Address,Facility_Type
245,Stannaway,Shielton Athletic F.C,United Churches Football League,53.32068665,-6.30068917285273,"Stannaway, Kimmage B Ed, Dublin 12, Dublin, Co...",Soccer
246,Walkinstown,Castle Celtic,United Churches Football League,53.3216133,-6.3301177,"Walkinstown, Walkinstown B Ed, Dublin 12, Dubl...",Soccer
247,St Annes,F.C Ailesbury Utd.,United Churches Football League/Amateur Footba...,53.2651078,-6.3609563,"Saint Anne'S, Bohernabreena Ed, Dublin 24, Sou...",Soccer
248,Belcamp,Sporting Metro F.C.,United Churches Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer
249,St Annes,Clontarf F.C (Seniors),United Churches Football League,53.2651078,-6.3609563,"Saint Anne'S, Bohernabreena Ed, Dublin 24, Sou...",Soccer


**What locations were not found?**

In [891]:
enriched_dcc[enriched_dcc.Latitude==0]

,PARK,AREA,CLUBNAME,LEAGUE,Latitude,Longitude
29,FR. COLLINS,NORTH CENTRAL,WYTELEAF UNITED F.C.,AMATEUR FOOTBALL LEAGUE/ATHLETIC UNION LEAGUE,0.0,0.0
68,MC AULEY,NORTH CENTRAL,ST. BRENDANS UNITED,ATHLETIC UNION LEAGUE/ NORTH DUBLIN SCHOOLBO...,0.0,0.0
80,FR. COLLINS,NORTH CENTRAL,CLARE HALL CELTIC,DUBLIN DISTRICT SCHOOLBOYS LEAGUE,0.0,0.0
109,MC AULEY,NORTH CENTRAL,ST. PAULS ARTANE,"DUBLIN DISTRICT SCHOOLBOYS LEAGUE,,ATHLETIC UN...",0.0,0.0
111,FR. COLLINS,NORTH CENTRAL,TRINITY/DONAGHMEDE F.C.,"DUBLIN DISTRICT SCHOOLBOYS LEAGUE,LEINSTER SE...",0.0,0.0
129,FR. COLLINS,NORTHWEST,TRINITY GAELS,GAELIC ATHLETIC ASSOCIATION,0.0,0.0
208,FR. COLLINS,NORTH CENTRAL,MID SUTTON F.C,"NORTH DUBLIN SCHOOLBOYS LEAGUE,LEINSTER SENIOR...",0.0,0.0
227,FR. COLLINS,NORTH CENTRAL,SCHALKE ROVERS,UNITED CHURCHES FOOTBALL LEAGUE,0.0,0.0
239,FR. COLLINS,NORTH CENTRAL,SEAVIEW CELTIC,UNITED CHURCHES FOOTBALL LEAGUE,0.0,0.0


FR. COLLINS and MC AULEY

**Sanity Check for coordinates**

In [892]:
enriched_dcc.Latitude.unique()

array([53.3854709 , 53.3175917 , 53.4068274 , 53.4009603 , 53.3948612 ,
       53.36726325, 53.9863743 , 53.3410089 , 53.3563042 , 53.3977132 ,
       53.341944  , 53.2794678 , 53.3216133 ,  0.        , 53.31905805,
       53.361111  , 53.4223167 , 53.3840936 , 53.3380806 , 53.54157665,
       53.4040962 , 53.3358185 , 53.3829908 , 53.3898277 , 53.3981308 ,
       53.40234405, 53.5850302 , 53.3628457 , 53.3320387 , 53.3201888 ,
       53.3021198 , 53.3857502 , 53.3719591 , 53.32068665, 53.2475659 ,
       53.3210024 , 53.3803639 , 53.333914  , 53.2748878 , 53.3746929 ,
       53.386264  , 53.3870122 , 53.40424065, 53.3344399 , 53.3392107 ,
       53.34347825, 53.3099564 , 53.3337263 , 53.3276869 , 53.394585  ,
       53.3870818 , 53.3735101 , 53.38171705, 53.4020707 , 53.3663095 ,
       53.3989894 , 53.29428765, 53.38606505, 53.3854698 , 53.309439  ,
       53.32998585])

In [893]:
enriched_dcc.Longitude.unique()

array([-6.26080915, -6.2330531 , -6.19692619, -6.19403178, -6.1635775 ,
       -6.30703593, -7.3188437 , -6.3400557 , -6.2289594 , -6.2225083 ,
       -6.226389  , -6.1707722 , -6.3301177 ,  0.        , -6.23539136,
       -6.2279928 , -6.26304854, -6.3003502 , -6.2221223 , -6.22770963,
       -6.31930216, -6.3541441 , -6.1889192 , -6.3068943 , -6.3120612 ,
       -6.28712461, -6.2888697 , -6.2366557 , -6.3033135 , -6.3260332 ,
       -6.29144609, -6.24044189, -6.3322962 , -6.30068917, -6.1202158 ,
       -6.29130368, -6.28746076, -6.3183767 , -6.1425269 , -6.2197423 ,
       -6.18738019, -6.1888673 , -6.2726355 , -6.3044476 , -6.2164504 ,
       -6.32916531, -6.2631069 , -6.29534399, -6.23656997, -6.225759  ,
       -6.2180884 , -6.310396  , -6.24575159, -6.2053385 , -6.2347491 ,
       -6.3114986 , -6.17281714, -6.15047013, -6.1603965 , -6.2107616 ,
       -6.27618443])

Values look in range 53 and -6 which looks good

In [67]:
## add facility type as Soccer
enriched_dcc["Facility_Type"] = "Soccer"
# DCC content is all upper case > change to lower case
enriched_dcc = enriched_dcc.apply(lambda x: x.astype(str).str.title())
#change columns names to lower case
enriched_dcc.columns = map(str.title, enriched_dcc.columns)
# change all names into Location, and change the field "location" in FCC to "Town"
enriched_dcc.rename(columns={"Park":"Location"},inplace=True)

In [54]:
#getting address info for fcc
enriched_fcc= pd.DataFrame(fccPitches)

enriched_fcc["cords"] = enriched_fcc["LAT"].map(str) +","+ enriched_fcc["LONG"].map(str)
enriched_fcc['Address']  = enriched_fcc["cords"].apply(lambda x:geolocator.reverse(x).address) 


In [ ]:
enriched_fcc.drop(columns="cords",inplace=True)
enriched_fcc.drop(columns="Location",inplace=True)

In [152]:
enriched_fcc['Facility_Type']= enriched_fcc['Facility_Type'].apply(lambda x: x.replace('pitches',''))
enriched_fcc['Facility_Type']= enriched_fcc['Facility_Type'].apply(lambda x: x.replace('Pitches',''))

In [153]:
enriched_fcc.head()

,Facility_Type,Location,Latitude,Longitude,Address
0,All weather,Balbriggan Town Park,53.6049596246817,-6.18235291959051,"Glebe North Football Club, Market Green, Balbr..."
1,All weather,Balheary Reservoir,53.4727096370551,-6.22301521551813,"Swords celtic all weather pitch, Balheary Road..."
2,All weather,Town Park,53.5771135903791,-6.11107205744599,"The Kybe, Beau Piers, Townparks, Skerries ED, ..."
3,All weather,St. Mologa's Park,53.6176672458903,-6.18936794084573,"St. Molaga's National School, Lambeecher, Tank..."
4,Basketball Court,Seagrange Park,53.3966674985382,-6.13535180348378,"Progressive Credit Union, Grange Road, Baldoyl..."


In [158]:
#duplicates
enriched_fcc[enriched_fcc.duplicated(["Location","Facility_Type"])]

,Facility_Type,Location,Latitude,Longitude,Address
11,GAA,Balheary Park,53.4660349552282,-6.21611968889385,"Balheary pitch 1, Lissenhall Road, Balheary De..."
12,GAA,Balheary Park,53.467348630642,-6.21538771007825,"Balheary pitch 2, Lissenhall Road, Balheary De..."
14,GAA,Hartstown Park,53.3953231958123,-6.41292838835715,"Hartstown Community School, Hartstown Road, Ha..."
33,Seven-a-side,Hartstown Park,53.3963384827422,-6.41292397066233,"Blakestown, Blanchardstown-Blakestown ED, Clon..."
34,Seven-a-side,Hartstown Park,53.3962424394402,-6.41037074601988,"Blakestown, Blanchardstown-Blakestown ED, Dubl..."
37,Seven-a-side,St. Mologa's Park,53.6156257951025,-6.18391354720681,"Lifeboat House, Baths Road, Tankardstown, Balb..."
44,Soccer,Balbriggan Town Park,53.604642797328,-6.18133609646353,"Glebe North Football Club, Market Green, Balbr..."
47,Soccer,Newbridge Demesne,53.4898838481802,-6.17645552041318,"Cricket Ground, Hearse Road, Newbridge Demesne..."
48,Soccer,Newbridge Demesne,53.4892921655101,-6.17685013930942,"Donabate Catholic Parish, Cobbs Lane, Lanestow..."
49,Soccer,Newbridge Demesne,53.4886223910085,-6.17706018664759,"Newbridge Demesne, Donabate ED, Donabate, Fing..."


In [ ]:
#change columns names to lower case
enriched_fcc.columns = map(str.title, fccPitches.columns)


In [124]:
#remove duplicates
enriched_fcc.drop_duplicates(subset=["Address","Facility_Type"],inplace=True)

In [65]:

enriched_fcc.rename(columns={"Lat":"Latitude","Long":"Longitude"},inplace=True)


In [66]:
enriched_fcc.rename(columns={"Facility_Name":"Location"},inplace=True)

## Final DATA Model


In [129]:
final_df= pd.concat([enriched_dcc,enriched_dlr,enriched_fcc],
                    names=["Location","Latitude","Longitude","Size","Club_Name","League","Facility_Type"]
                   ,sort=False,ignore_index=True)



In [130]:
final_df.head()

,Location,Clubname,League,Latitude,Longitude,Address,Facility_Type,Size
0,Albert College,Drumcondra F.C (Snr),Amateur Football League,53.3854709,-6.26080914918107,"Albert College (A), Albert College Avenue, Alb...",Soccer,NaN
1,Albert College,Glasnevin Afc,Amateur Football League,53.3854709,-6.26080914918107,"Albert College (A), Albert College Avenue, Alb...",Soccer,NaN
2,Beechill,Ballsbridge Fc,Amateur Football League,53.3175917,-6.2330531,"Beechill Ave (Beaver Row), Beech Hill Estate, ...",Soccer,NaN
3,Belcamp,Newtown Celtic,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN
4,Belcamp,Vianney Boys,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN


## Data Quality

### Finding Duplicates

In [132]:
final_df[final_df.duplicated(subset=["Address","Facility_Type","Size"])]

,Location,Clubname,League,Latitude,Longitude,Address,Facility_Type,Size
1,Albert College,Glasnevin Afc,Amateur Football League,53.3854709,-6.26080914918107,"Albert College (A), Albert College Avenue, Alb...",Soccer,NaN
4,Belcamp,Vianney Boys,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN
5,Belcamp,Priorswood F.C Seniors,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN
6,Belcamp,Whites F.C,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN
7,Belcamp,Urban Celtic,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN
8,Belcamp,Clonshaugh Sporting Suazee,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,NaN
13,Kilmore,Vianney Boys,Amateur Football League,54.40024675,-6.55048274536792,"Kilmore, Armagh, County Armagh, Northern Irela...",Soccer,NaN
18,Ringsend,Irishtown F.C.,Amateur Football League,53.341944,-6.226389,"Ringsend, Pembroke East A Ed, Dublin 4, Dublin...",Soccer,NaN
20,St Annes,Chanel Sports & Social Club,Amateur Football League,53.2651078,-6.3609563,"Saint Anne'S, Bohernabreena Ed, Dublin 24, Sou...",Soccer,NaN
21,St Annes,Marino Athletic,Amateur Football League,53.2651078,-6.3609563,"Saint Anne'S, Bohernabreena Ed, Dublin 24, Sou...",Soccer,NaN


### Do we care about Clubnames?

### Checking for null values

In [133]:
final_df.isna().any()

Location         False
Clubname          True
League            True
Latitude         False
Longitude        False
Address          False
Facility_Type    False
Size              True
dtype: bool

we have null values in the extra fields but not in Location, Latitude, Longitude

### Remove White Spaces


we take a look at Locations since it is the most important field

In [137]:
final_df.sort_values(by="Location").Location.unique()

array(['Albert College', 'Alfie Byrne', 'Ardmore', 'Ashington',
       'Balbriggan Town Park', 'Balcurris Park', 'Balheary Park',
       'Balheary Reservoir', 'Ball Alley/Remount', 'Ballist Pit',
       'Balrothery Park', 'Bayside School', 'Beechill', 'Belcamp',
       'Benmadigan', 'Brickfields', 'Broomfiled', 'Bunting Road', 'Bushy',
       'Carrickhill Park', 'Cherry Orchard', 'Clonturk', 'Coolmine Park',
       'Corballis Road', 'Corduff Park', 'Coultry', 'Craoibhinn Park',
       "Danny's Pitches", 'Darndale', 'Dartry', 'Dolphin', 'Donaghmede',
       'Donnycarney', 'Dundaniel', 'Dunedin (jun)', 'Edenmore',
       'Ellenfield', 'Fairview', 'Farnham', 'Fr. Collins', 'Galtymore',
       'Glin Road', "Grace O'Malley Park", 'Greendale', 'Hartstown Park',
       'Hazelbury Park', 'Herbert', 'Holly Park', 'Hundson Road',
       'Hunters Run/lLittlep', 'Hyde Park', 'Irishtown', 'John Paul',
       'Johnstown', 'Kilbogget Park', 'Kildonan', 'Kilmore',
       "Lady's Well Park", 'Laurel Lo

we notice white spaces in the fields so we remove that

In [135]:
final_df =final_df.apply(lambda x: x.astype(str).str.strip())
final_df = final_df.replace(' +', ' ',regex=True).astype(str)


we notice several typos like:
    - 'Newbridge Demense', 'Newbridge Demesne'
    - 'St Annes', 'St. Annes'
    - "St. Catherine's Park", 'St. Catherines'
    - "St. Molaga's Park", "St. Mologa's Park"
so we unite that.

In [136]:
final_df.loc[final_df['Location'] =='Newbridge Demense', 'Location'] = 'Newbridge Demesne'
final_df.loc[final_df['Location'] =='St Annes', 'Location'] = 'St. Annes'
final_df.loc[final_df['Location'] =='St. Catherines', 'Location']  = 'St. Catherine\'s Park'
final_df.loc[final_df['Location'] =='St. Molaga\'s Park', 'Location']  = 'St. Mologa\'s Park'


In [138]:
final_df =final_df.apply(lambda x: x.astype(str).str.strip())

,Location,Clubname,League,Latitude,Longitude,Address,Facility_Type,Size
1,Albert College,Glasnevin Afc,Amateur Football League,53.3854709,-6.26080914918107,"Albert College (A), Albert College Avenue, Alb...",Soccer,nan
4,Belcamp,Vianney Boys,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,nan
5,Belcamp,Priorswood F.C Seniors,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,nan
6,Belcamp,Whites F.C,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,nan
7,Belcamp,Urban Celtic,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,nan
8,Belcamp,Clonshaugh Sporting Suazee,Amateur Football League,53.4068274,-6.19692618719913,"Belcamp, Priorswood B Ed, Dublin 17, Dublin, C...",Soccer,nan
13,Kilmore,Vianney Boys,Amateur Football League,54.40024675,-6.55048274536792,"Kilmore, Armagh, County Armagh, Northern Irela...",Soccer,nan
18,Ringsend,Irishtown F.C.,Amateur Football League,53.341944,-6.226389,"Ringsend, Pembroke East A Ed, Dublin 4, Dublin...",Soccer,nan
20,St. Annes,Chanel Sports & Social Club,Amateur Football League,53.2651078,-6.3609563,"Saint Anne'S, Bohernabreena Ed, Dublin 24, Sou...",Soccer,nan
21,St. Annes,Marino Athletic,Amateur Football League,53.2651078,-6.3609563,"Saint Anne'S, Bohernabreena Ed, Dublin 24, Sou...",Soccer,nan


In [588]:
#install for location 
! pip install geopy

    100% |████████████████████████████████| 102kB 894kB/s a 0:00:011
  Running setup.py bdist_wheel for geographiclib ... done
  Stored in directory: /Users/suaddarra/Library/Caches/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
